In [1]:
import gpu_tracking
import tifffile

In [7]:
vid = tifffile.imread("../emily_tracking/sample_vids/s_20.tif").astype("float32")

In [8]:
gpu_tracking.batch(vid, 9)

[src/execute_gpu.rs:346] wait_gpu_time = 2.2090204


array([[   0.       , 1396.9764   ,    5.4445252,   39.444527 ],
       [   0.       , 1222.1484   ,    4.9359045,   56.935905 ],
       [   0.       ,  373.26227  ,    5.784588 ,   84.78459  ],
       ...,
       [1999.       , 1059.4739   ,  502.4651   ,  421.4651   ],
       [1999.       ,   96.72125  ,  514.4057   ,  475.40573  ],
       [1999.       ,  697.91266  ,  503.5507   ,  481.5507   ]],
      dtype=float32)